# 02 - Simulating One Dimensional Harmonic Oscillators

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
force_constant = 1.0 # @param {type:"number"}
velocity_scale = 1  # @param {type:"number"}
integrator = "Euler"  # @param ["Velocity Verlet", "Euler"]
dt = 0.01  # @param {type:"number"}
nsteps = 1000  # @param {type:"integer"}
frame_stride = 1 # @param {type:"integer"}
total_time = nsteps * dt

In [ ]:
position_a = 0.
velocity_a = 1.0 * velocity_scale
mass_a = 1.
position_b = 5.
velocity_b = -1.0 * velocity_scale
mass_b = 1.

kinetic = 0.5 * mass_a * velocity_a**2 + 0.5 * mass_b * velocity_b**2
potential = 0.5 * force_constant * (position_a - position_b)**2    

time = np.arange(0, total_time, dt)

# --- Analytical solution ---
omega = np.sqrt(2 * force_constant / mass_a)  # sqrt(2k/m)

# Initial relative position and velocity
x_r0 = position_a - position_b  # -5
v_r0 = velocity_a - velocity_b  # 2

# Time-independent center of mass
x_cm = (position_a + position_b) / 2         # 2.5
v_cm = (velocity_a + velocity_b) / 2         # 0

# Relative motion
x_r_t = x_r0 * np.cos(omega * time) + (v_r0 / omega) * np.sin(omega * time)
v_r_t = -x_r0 * omega * np.sin(omega * time) + v_r0 * np.cos(omega * time)

# Analytical positions and velocities
xA_analytic = x_cm + 0.5 * x_r_t
xB_analytic = x_cm - 0.5 * x_r_t
vA_analytic = v_cm + 0.5 * v_r_t
vB_analytic = v_cm - 0.5 * v_r_t

# Energy
E_kin_analytic = 0.5 * mass_a * vA_analytic**2 + 0.5 * mass_b * vB_analytic**2
E_pot_analytic = 0.5 * force_constant * (xA_analytic - xB_analytic)**2
E_total_analytic = E_kin_analytic + E_pot_analytic

trajectory_a = []
trajectory_b = []
energy = []

# Run the simulation loop
for _ in time:
    trajectory_a.append((position_a,velocity_a))
    trajectory_b.append((position_b,velocity_b))
    energy.append((kinetic,potential,kinetic + potential))

    # Compute forces
    force_a = -force_constant * (position_a - position_b)
    force_b = -force_constant * (position_b - position_a)

    if integrator == "Euler":
        # Update positions and velocities using Euler method
        position_a += velocity_a * dt
        position_b += velocity_b * dt
        velocity_a += force_a / mass_a * dt
        velocity_b += force_b / mass_b * dt
    elif integrator == "Velocity Verlet":
        # Update positions and velocities using Velocity Verlet
        position_a += velocity_a * dt + 0.5 * force_a / mass_a * dt**2
        position_b += velocity_b * dt + 0.5 * force_b / mass_b * dt**2
        new_force_a = -force_constant * (position_a - position_b)
        new_force_b = -force_constant * (position_b - position_a)
        velocity_a += 0.5 * (force_a + new_force_a) / mass_a * dt
        velocity_b += 0.5 * (force_b + new_force_b) / mass_b * dt

    kinetic = 0.5 * mass_a * velocity_a**2 + 0.5 * mass_b * velocity_b**2
    potential = 0.5 * force_constant * (position_a - position_b)**2    

# Convert lists to numpy arrays for easier handling
time = np.array(time)
trajectory_a = np.array(trajectory_a)
trajectory_b = np.array(trajectory_b)
energy = np.array(energy)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# --- Create subplot layout ---
fig = make_subplots(
    rows=3, cols=2,
    specs=[[{}, {}],
           [{}, {}],
           [{"colspan": 2}, None]],
    subplot_titles=[
        "Particle Positions",
        "Position A vs Time",
        "Velocity A vs Time",
        "Phase Space (Position A vs Velocity A)",
        "Energies vs Time"
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.15
)
# --- Initial numerical traces (frame 0) ---
fig.add_trace(go.Scatter(x=[trajectory_a[0, 0]], y=[0], mode='markers',
                         marker=dict(size=15, color='blue'), name='A'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=[trajectory_b[0, 0]], y=[0], mode='markers',
                         marker=dict(size=15, color='red'), name='B'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=xA_analytic, y=time, mode='lines',
                         line=dict(color='lightgrey'), showlegend=False),
              row=1, col=2)
fig.add_trace(go.Scatter(x=[trajectory_a[0, 0]], y=[time[0]], mode='lines',
                         line=dict(color='blue'), name='xA(t)'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=time, y=vA_analytic, mode='lines',
                         line=dict(color='lightgrey'), showlegend=False),
              row=2, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[trajectory_a[0, 1]], mode='lines',
                         line=dict(color='green'), name='vA(t)'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=xA_analytic, y=vA_analytic, mode='markers',
                         line=dict(color='lightgrey'), showlegend=False),
              row=2, col=2)
fig.add_trace(go.Scatter(x=[trajectory_a[0, 0]], y=[trajectory_a[0, 1]], mode='markers',
                         line=dict(color='purple'), name='Phase A'),
              row=2, col=2)

fig.add_trace(go.Scatter(x=time, y=E_kin_analytic, mode='lines', line=dict(color='lightgrey', dash='dot'), showlegend=False),
              row=3, col=1)
fig.add_trace(go.Scatter(x=time, y=E_pot_analytic, mode='lines', line=dict(color='lightgrey', dash='dot'), showlegend=False),
              row=3, col=1)
fig.add_trace(go.Scatter(x=time, y=E_total_analytic, mode='lines', line=dict(color='lightgrey', dash='dot'), showlegend=False),
              row=3, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[energy[0, 0]], mode='lines',
                         line=dict(color='orange'), name='KE'),
              row=3, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[energy[0, 1]], mode='lines',
                         line=dict(color='blue'), name='PE'),
              row=3, col=1)
fig.add_trace(go.Scatter(x=[time[0]], y=[energy[0, 2]], mode='lines',
                         line=dict(color='black'), name='Total E'),
              row=3, col=1)
        
# --- Animation frames ---
frames = []
for i in range(0, nsteps, frame_stride):
    frames.append(go.Frame(data=[
        # Row 1, Col 1: Particle positions
        go.Scatter(x=[trajectory_a[i, 0]], y=[0], mode='markers', marker=dict(size=15, color='blue')),
        go.Scatter(x=[trajectory_b[i, 0]], y=[0], mode='markers', marker=dict(size=15, color='red')),

        # Row 1, Col 2: Position vs Time (flipped)
        go.Scatter(x=xA_analytic, y=time, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=trajectory_a[:i, 0], y=time[:i], mode='lines', line=dict(color='blue')),

        # Row 2, Col 1: Velocity vs Time
        go.Scatter(x=time, y=vA_analytic, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time[:i], y=trajectory_a[:i, 1], mode='lines', line=dict(color='green')),

        # Row 2, Col 2: Phase space
        go.Scatter(x=xA_analytic, y=vA_analytic, mode='markers', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=trajectory_a[:i, 0], y=trajectory_a[:i, 1], mode='markers', line=dict(color='purple')),

        # Row 3, Col 1: Energies
        go.Scatter(x=time, y=E_kin_analytic, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time, y=E_pot_analytic, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time, y=E_total_analytic, mode='lines', line=dict(color='lightgrey'), showlegend=False),
        go.Scatter(x=time[:i], y=energy[:i, 0], mode='lines', line=dict(color='orange')),
        go.Scatter(x=time[:i], y=energy[:i, 1], mode='lines', line=dict(color='blue')),
        go.Scatter(x=time[:i], y=energy[:i, 2], mode='lines', line=dict(color='black')),

    ]))

# --- Attach frames ---
fig.frames = frames

# --- Layout and buttons ---
total_time = nsteps * dt
fig.update_layout(
    height=1000,
    width=1000,
    title={
    "text": "1D Spring Simulation: Numerical vs Analytical",
    "x": 0.5,             # Centered (try 0.5–0.8 for right alignment)
    "xanchor": "left"     # Align title's anchor point
    },
    showlegend=True,updatemenus=[
    dict(
        type="buttons",
        showactive=True,
        x=0,
        y=1.05,  # instead of 1.15
        xanchor="left",
        yanchor="bottom",
        direction="right",
        buttons=[
            dict(label="▶️ Slow", method="animate",
                 args=[None, {
                     "frame": {"duration": 1000, "redraw": True},
                     "fromcurrent": True,
                     "transition": {"duration": 0}
                 }]),
            dict(label="▶️ Medium", method="animate",
                 args=[None, {
                     "frame": {"duration": 200, "redraw": True},
                     "fromcurrent": True,
                     "transition": {"duration": 0}
                 }]),
            dict(label="▶️ Fast", method="animate",
                 args=[None, {
                     "frame": {"duration": 50, "redraw": True},
                     "fromcurrent": True,
                     "transition": {"duration": 0}
                 }]),
            dict(label="⏸ Pause", method="animate",
                 args=[[None], {
                     "mode": "immediate",
                     "frame": {"duration": 0, "redraw": False},
                     "transition": {"duration": 0}
                 }])
        ]
    )
]
)

# Compute shared x_A range
x_min = min(xA_analytic.min(), trajectory_a[:, 0].min())
x_max = max(xA_analytic.max(), trajectory_a[:, 0].max())
fig.update_xaxes(title="x_A", range=[x_min, x_max], row=1, col=2)
fig.update_xaxes(title="x_A", range=[x_min, x_max], row=2, col=2)

# Compute shared v_A range
v_min = min(vA_analytic.min(), trajectory_a[:, 1].min())
v_max = max(vA_analytic.max(), trajectory_a[:, 1].max())
fig.update_yaxes(title="v_A", range=[v_min, v_max], row=2, col=1)
fig.update_yaxes(title="v_A", range=[v_min, v_max], row=2, col=2)

# --- Axis labels and ranges ---
fig.update_xaxes(title="Position", row=1, col=1)
fig.update_yaxes(showticklabels=False, row=1, col=1)

fig.update_xaxes(title="x_A", row=1, col=2)
fig.update_yaxes(title="Time", range=[0, total_time], row=1, col=2)

fig.update_xaxes(title="Time", range=[0, total_time], row=2, col=1)
fig.update_yaxes(title="v_A", row=2, col=1)

fig.update_xaxes(title="x_A", row=2, col=2)
fig.update_yaxes(title="v_A", row=2, col=2)

fig.update_xaxes(title="Time", range=[0, total_time], row=3, col=1)
fig.update_yaxes(title="Energy", row=3, col=1)

# --- Show the animated figure ---
fig.show()


In [ ]:
phi = np.linspace(0, 2 * np.pi, 100)
potential_1 = (1 + np.cos(2*phi+np.pi)) 
potential_2 = (1 + np.cos(phi))
plt.plot(phi, potential_1)
plt.plot(phi, potential_2)
plt.plot(phi, potential_1 + potential_2)

In [ ]:
phi = np.pi 
dphidt = 0.1
potential = (1 + np.cos(2*phi+np.pi)) + (1 + np.cos(phi))
inertia = 1.

dt = 0.1
nsteps = 100
total_time = nsteps * dt
time = np.arange(0, total_time, dt)

trajectory = []
energy = []

# Run the simulation loop
for _ in time:
    trajectory.append((phi,dphidt))
    energy.append(potential)

    # Compute forces
    force = ( 2 * np.sin(2*phi+np.pi) + np.sin(phi) )

    # Update positions and velocities using Velocity Verlet
    phi += dphidt * dt + 0.5 * force / inertia * dt**2
    if phi > 2*np.pi:
        phi -= 2*np.pi
    elif phi < 0:
        phi += 2*np.pi
    new_force = ( 2 * np.sin(2*phi+np.pi) + np.sin(phi) )
    dphidt += 0.5 * (force + new_force) / inertia * dt

    potential = (1 + np.cos(2*phi+np.pi)) + (1 + np.cos(phi))    

# Convert lists to numpy arrays for easier handling
trajectory = np.array(trajectory)
energy = np.array(energy)

In [ ]:
plt.plot(time, trajectory[:,0])